In [10]:
from functionUtils import *
from sklearn.model_selection import KFold,StratifiedKFold,GroupKFold
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm_notebook, tnrange
from scipy.stats import ks_2samp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime
import gc
import lightgbm as lgb
import xgboost as xgb
import catboost as catb
import os
warnings.filterwarnings("ignore")
fea_path = './datasets/'
%matplotlib inline

In [2]:
df_data = pd.read_csv(fea_path+'df_data.csv',dtype={'first_active_month':np.str})
df_train_test_additional_features = pd.read_csv(fea_path+'df_train_test_features_additional.csv')
df_additional_features = pd.read_csv(fea_path+'df_additional_features.csv')

df_data = df_data.merge(df_train_test_additional_features,on='card_id',how='left')
df_data = df_data.merge(df_additional_features,on='card_id',how='left')

In [3]:
path = './datasets/feature2/'
sublist = os.listdir(path)

for sub in sublist:
    df = pd.read_csv(path+sub)
    df_data = df_data.merge(df,on='card_id',how='left')

In [4]:
df_cate_merchantCate_fm = pd.read_csv('./datasets/cate_fm/df_hist_new_cate_merchantCate_fm_feat.csv')
df_cate_merchant_fm = pd.read_csv('./datasets/cate_fm/df_hist_new_cate_merchant_fm_feat.csv')

df_data = df_data.merge(df_cate_merchantCate_fm,on='card_id',how='left')
df_data = df_data.merge(df_cate_merchant_fm,on='card_id',how='left')

del df_cate_merchantCate_fm,df_cate_merchant_fm
gc.collect()

53

In [5]:
df_card_merchant_statics = pd.read_csv(fea_path+'df_card_merchant_statics.csv')
df_card_merchant_vec = pd.read_csv(fea_path+'df_card_merchant_vec.csv')
df_card_city_statics = pd.read_csv(fea_path+'df_card_city_statics.csv')

df_data = df_data.merge(df_card_merchant_vec,on='card_id',how='left')
df_data = df_data.merge(df_card_merchant_statics,on='card_id',how='left')
df_data = df_data.merge(df_card_city_statics,on='card_id',how='left')

del df_card_merchant_statics,df_card_merchant_vec,df_card_city_statics
gc.collect()

42

In [6]:
df_nmf_card_merCate_features = pd.read_csv(fea_path+'df_nmf_card_merCate_features.csv')
df_nmf_card_city_features = pd.read_csv(fea_path+'df_nmf_card_city_features.csv')
df_card_merchant_features = pd.read_csv(fea_path+'df_card_merchant_features.csv')
df_cardid_vec = pd.read_csv(fea_path+'df_cardid_vec.csv')
# # df_card_merchant_vec1 = pd.read_csv('./datasets/df_card_merchant_vec1.csv')
# # df_f1_f2_f3_vec = pd.read_csv('./datasets/df_f1_f2_f3_vec.csv')

df_data = df_data.merge(df_nmf_card_merCate_features,on='card_id',how='left')
df_data = df_data.merge(df_nmf_card_city_features,on='card_id',how='left')
df_data = df_data.merge(df_card_merchant_features,on='card_id',how='left')
df_data = df_data.merge(df_cardid_vec,on='card_id',how='left')

del df_nmf_card_merCate_features,df_nmf_card_city_features,df_card_merchant_features,df_cardid_vec
gc.collect()

56

In [7]:
df_train = df_data[df_data.is_test==0]
df_test = df_data[df_data.is_test==1]

In [8]:
label = ['is_outlier']
df_train['is_outlier'] = (df_train.target<-30).astype(np.int)

dropCols = ['card_id','first_active_month','is_outlier','is_test','target','purchase_date','merchant_id']
tr_features = [_f for _f in df_train.columns if _f not in dropCols and df_train[_f].dtype!='object']

In [ ]:
def modelKFlodClf(df_train,df_test,clf):
    NFOLDS = 5
    kfold = StratifiedKFold(n_splits=NFOLDS,shuffle=False,random_state=20)

    ntrain = df_train.shape[0]
    ntest = df_test.shape[0]
    oof_train_pred = np.zeros((ntrain,))
    oof_test_pred = np.zeros((ntest,))
    oof_test_pred_skf = np.empty((NFOLDS, ntest))
    for foldIndex,(dev_index,val_index) in enumerate(kfold.split(df_train,df_train['is_outlier'])):
        print("............第%s折..........."%(foldIndex+1))
        x_dev = df_train[tr_features].iloc[dev_index]
        y_dev = df_train[label].iloc[dev_index]
        x_val = df_train[tr_features].iloc[val_index]
        y_val = df_train[label].iloc[val_index]
        clf.fit(x_dev, y_dev,eval_set=[(x_dev,y_dev),(x_val,y_val)],early_stopping_rounds=50,verbose=100)
        oof_test_pred_skf[foldIndex,:] = clf.predict_proba(df_test[tr_features],num_iteration=clf.best_iteration_)[:,0]
        oof_train_pred[val_index] = clf.predict_proba(x_val,num_iteration=clf.best_iteration_)[:,0]
    oof_test_pred[:] = oof_test_pred_skf.mean(axis=0)
    return oof_train_pred.reshape(-1,1),oof_test_pred.reshape(-1,1)

lgb_clf_params = {
    'num_leaves': 31,
    'min_data_in_leaf': 30,
    'objective': 'binary',
    'n_estimators':2000,
    'max_depth': 8,
    'learning_rate': 0.01,
    "min_child_samples": 20,
    'class_weight':'balanced',#平衡样本
    "boosting": "gbdt",
    "feature_fraction": 0.9,
#     "bagging_freq": 1,
#     "bagging_fraction": 0.9,
#     "bagging_seed": 11,
    "metric": ['auc', 'binary_logloss'],
    "lambda_l1": 10,
    "verbosity": -1,
    "nthread": 16,
    "random_state": 2019
}

lgb_clf = lgb.LGBMClassifier(**lgb_clf_params)
lgb_train_pred,lgb_test_pred = modelKFlodClf(df_train,df_test,lgb_clf)

In [ ]:
df_train['lgb_outlier_pred'] = 1 - lgb_train_pred
df_test['lgb_outlier_pred'] = 1 - lgb_test_pred

In [ ]:
%%time

def modelKFoldClf(df_train,df_test,model):
    
    NFOLDS = 5
    kfold = StratifiedKFold(n_splits=NFOLDS,shuffle=False,random_state=20)

    ntrain = df_train.shape[0]
    ntest = df_test.shape[0]
    oof_train_pred = np.zeros((ntrain,))
    oof_test_pred = np.zeros((ntest,))
    oof_test_pred_skf = np.empty((NFOLDS, ntest))
    for foldIndex,(dev_index,val_index) in enumerate(kfold.split(df_train,df_train['is_outlier'])):
        print("............第%s折..........."%(foldIndex+1))
        x_dev = df_train[tr_features].iloc[dev_index]
        y_dev = df_train[label].iloc[dev_index]
        x_val = df_train[tr_features].iloc[val_index]
        y_val = df_train[label].iloc[val_index]
        model.fit(x_dev, y_dev,eval_set=[(x_dev,y_dev),(x_val,y_val)],use_best_model=True,early_stopping_rounds=100,verbose=100)
        oof_test_pred_skf[foldIndex,:] = model.predict_proba(df_test[tr_features])[:,0]
        oof_train_pred[val_index] = model.predict_proba(x_val)[:,0]
    oof_test_pred[:] = oof_test_pred_skf.mean(axis=0)
    score = np.sqrt((np.sum(np.square(oof_train_pred - df_train[label].values.reshape(-1,)))/ntrain))
    return oof_train_pred.reshape(-1,1),oof_test_pred.reshape(-1,1)

ratio = sum(df_train['is_outlier']==0)/sum(df_train['is_outlier']==1)

print("ratio = "+str(ratio))

cat_params = {
    'n_estimators':5000,
    'learning_rate':0.01,
    'max_depth':8,
    'loss_function':'Logloss',
#     'eval_metric':'F1',
    'eval_metric':'AUC',
    'scale_pos_weight':ratio,
    'logging_level':'Verbose',
    'random_state':40,
    'bagging_temperature':0.8,
#     'l2_leaf_reg':45,
    'od_type':'Iter',
    'thread_count':16
}

cat_est = catb.CatBoostClassifier(**cat_params)
oof_train_pred,oof_test_pred = modelKFoldClf(df_train,df_test,cat_est)
cat_est.predict_proba

In [ ]:
df_train['cat_outlier_pred'] = 1 - oof_train_pred
df_test['cat_outlier_pred'] = 1 - oof_test_pred

In [ ]:
df_train.sort_values(by=['cat_outlier_pred'],ascending=False)[['card_id','is_outlier','cat_outlier_pred']]

In [ ]:
gc.collect()

In [ ]:
%%time

def modelKFoldClf(df_train,df_test,model):
    
    NFOLDS = 5
    kfold = StratifiedKFold(n_splits=NFOLDS,shuffle=False,random_state=2018)

    ntrain = df_train.shape[0]
    ntest = df_test.shape[0]
    oof_train_pred = np.zeros((ntrain,))
    oof_test_pred = np.zeros((ntest,))
    oof_test_pred_skf = np.empty((NFOLDS, ntest))
    for foldIndex,(dev_index,val_index) in enumerate(kfold.split(df_train,df_train['is_outlier'])):
        print("............第%s折..........."%(foldIndex+1))
        x_dev = df_train[tr_features].iloc[dev_index]
        y_dev = df_train[label].iloc[dev_index]
        x_val = df_train[tr_features].iloc[val_index]
        y_val = df_train[label].iloc[val_index]
        model.fit(x_dev, y_dev,eval_set=[(x_dev,y_dev),(x_val,y_val)],early_stopping_rounds=50,verbose=100)
        oof_test_pred_skf[foldIndex,:] = model.predict_proba(df_test[tr_features])[:,0]
        oof_train_pred[val_index] = model.predict_proba(x_val)[:,0]
    oof_test_pred[:] = oof_test_pred_skf.mean(axis=0)
    return oof_train_pred.reshape(-1,1),oof_test_pred.reshape(-1,1)

ratio = sum(df_train['is_outlier']==0)/sum(df_train['is_outlier']==1)
print("ratio = "+str(ratio))

xgb_params = {
    'objective': 'binary:logistic',
    'booster': 'gbtree',
    'learning_rate': 0.01,
    'n_estimators':5000,
    'scale_pos_weight':ratio,
    'max_depth': 8,
    "eval_metric": ['auc', 'logloss'],
    'gamma' : 1.45,
    'alpha': 0.1,
    'lambda': 0.3,
    'subsample': 0.9,
    'colsample_bytree': 0.054,
    'colsample_bylevel': 0.50,
    'random_state': 2018
}
xgb_est = xgb.XGBClassifier(**xgb_params)
oof_train_pred,oof_test_pred = modelKFoldClf(df_train,df_test,xgb_est)

ratio = 90.48935206162211
............第1折...........
[0]	validation_0-auc:0.910632	validation_0-logloss:0.687383	validation_1-auc:0.828046	validation_1-logloss:0.687427
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 50 rounds.
[100]	validation_0-auc:0.970868	validation_0-logloss:0.393896	validation_1-auc:0.895837	validation_1-logloss:0.398036
[200]	validation_0-auc:0.979397	validation_0-logloss:0.288058	validation_1-auc:0.897713	validation_1-logloss:0.294999
[300]	validation_0-auc:0.985142	validation_0-logloss:0.235168	validation_1-auc:0.899367	validation_1-logloss:0.244545
[400]	validation_0-auc:0.989436	validation_0-logloss:0.202224	validation_1-auc:0.900085	validation_1-logloss:0.213832
[500]	validation_0-auc:0.992818	validation_0-logloss:0.178758	validation_1-auc:0.900276	validation_1-logloss:0.19204
[600]	validation_0-auc:0.99515	validation_0-logloss:0.160533	validation_1-auc

In [ ]:
df_train['xgb_outlier_pred'] = 1 - oof_train_pred
df_test['xgb_outlier_pred'] = 1 - oof_test_pred

In [ ]:
df_train.sort_values(by=['xgb_outlier_pred'],ascending=False)[['card_id','is_outlier','xgb_outlier_pred']]

In [ ]:
df_train[['card_id','xgb_outlier_pred']].to_csv('./datasets/class/df_xgb_train.csv',index=False)
df_test[['card_id','xgb_outlier_pred']].to_csv('./datasets/class/df_xgb_test.csv',index=False)


In [ ]:
abnormal_test = df_test.shape[0]*sum(df_train['is_outlier']==1)/df_train.shape[0]
print("测试集约有:%s异常值"%abnormal_test)

In [ ]:
##########

In [ ]:
outlier_cols = [_f for _f in df_train.columns if 'outlier_pred' in _f]
df_class_train = df_train[['card_id','is_outlier']+outlier_cols]
df_class_test = df_test[['card_id']+outlier_cols]

df_lgb_train = pd.read_csv('./datasets/class/lgb_classifier_train_0.0446_0.8994.csv')
df_lgb_test = pd.read_csv('./datasets/class/lgb_classifier_test_0.0446_0.8994.csv')

df_cat_train = pd.read_csv('./datasets/class/catboost_classifier_train_0.8934.csv')
df_cat_test = pd.read_csv('./datasets/class/catboost_classifier_test_0.8934.csv')

# df_lgb1_train = pd.read_csv('./datasets/class/lightgbm_classifier_train_0.8994_0.0446.csv')
# df_lgb1_test = pd.read_csv('./datasets/class/lightgbm_classifier_test_0.8994_0.0446.csv')


df_class_train = df_class_train.merge(df_lgb_train,on='card_id',how='left')
df_class_test = df_class_test.merge(df_lgb_test,on='card_id',how='left')

df_class_train = df_class_train.merge(df_cat_train,on='card_id',how='left')
df_class_test = df_class_test.merge(df_cat_test,on='card_id',how='left')

# df_class_train = df_class_train.merge(df_lgb1_train,on='card_id',how='left')
# df_class_test = df_class_test.merge(df_lgb1_test,on='card_id',how='left')


##分类器文件保存
df_class_train.to_csv('./datasets/class/df_class_train.csv',index=False)
df_class_test.to_csv('./datasets/class/df_class_test.csv',index=False)

In [ ]:
df_class_train.head()

#### Stacking Classifier

In [ ]:
from sklearn.linear_model import LogisticRegression

NFOLDS = 5
# kfold = KFold(n_splits=NFOLDS,shuffle=False,random_state=5)
kfold = StratifiedKFold(n_splits=NFOLDS,shuffle=False,random_state=20)
label = ['is_outlier']
tr_features = [_f for _f in df_class_train.columns if _f not in ['card_id','is_outlier']]

ntrain = df_class_train.shape[0]
ntest = df_class_test.shape[0]
oof_train_pred = np.zeros((ntrain,))
oof_test_pred = np.zeros((ntest,))
oof_test_pred_skf = np.empty((NFOLDS, ntest))
for foldIndex,(dev_index,val_index) in enumerate(kfold.split(df_class_train,df_class_train['is_outlier'])):
    clf = LogisticRegression(class_weight='balanced',random_state=42)
    x_dev = df_class_train[tr_features].iloc[dev_index]
    y_dev = df_class_train[label].iloc[dev_index]
    x_val = df_class_train[tr_features].iloc[val_index]
    y_val = df_class_train[label].iloc[val_index]
    clf.fit(x_dev.values,y_dev.values)
    oof_test_pred_skf[foldIndex,:] = clf.predict_proba(df_class_test[tr_features].values)[:,1]
    oof_train_pred[val_index] = clf.predict_proba(x_val.values)[:,1]

oof_test_pred[:] = oof_test_pred_skf.mean(axis=0)

score = np.sqrt((np.sum(np.square(oof_train_pred - df_class_train[label].values.reshape(-1,)))/ntrain))

df_Log_sub = pd.DataFrame(data=df_class_test['card_id'].values,columns=['card_id'])
df_Log_sub['log_outlier_pred'] = oof_test_pred
print("score = %s"%score)
df_Log_sub.sort_values(by=['log_outlier_pred'],ascending=False,inplace=True)

In [ ]:
topk = 100
outlier_cols = [_f for _f in df_class_train.columns if _f not in ['card_id','is_outlier']]

aggs = set(df_class_train.card_id)

for col in outlier_cols:
    df = df_class_train.sort_values(by=col,ascending=False)[['card_id',col]].iloc[:topk,]
    aggs = set(aggs) & set(df['card_id'])

print(len(aggs))
df = df_class_train[df_class_train['card_id'].isin(list(aggs))][['card_id','is_outlier']]
accuracy = df[df.is_outlier==1].shape[0]/len(aggs)

print("Accuracy: %s"%accuracy)

In [ ]:
df_class_train[df_class_train['card_id'].isin(list(aggs))]

In [ ]:
### testSets
df_temp = pd.read_csv('./classify.csv').sort_values(by=['outlier_rate'],ascending=False)
temp_list = list(df_temp.iloc[:topk,]['card_id'])

outlier_cols = [_f for _f in df_class_test.columns if _f not in ['card_id','target']]
aggs = set(temp_list)

for col in outlier_cols:
    df = df_class_test.sort_values(by=col,ascending=False)[['card_id',col]].iloc[:topk,]
    aggs = set(aggs) & set(df['card_id'])
aggs = set(aggs) & set(df_Log_sub['card_id'])
print(len(aggs))

In [ ]:
df_best_sub = pd.read_csv('./submission/df_stackingII_blend_sub.csv')
df_best_sub[df_best_sub['card_id'].isin(list(aggs))]

In [ ]:
df_best_sub.ix[df_best_sub['card_id'].isin(list(aggs)),['target']] = -33.21928095

In [ ]:
df_best_sub.to_csv('./submission/df_best_sub_with_21outliers.csv',index=False)

In [ ]:
plt.hist(df_best_sub.target,bins=200)